In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from IPython.display import Markdown
from ISLP import load_data
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis,
    QuadraticDiscriminantAnalysis,
)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

weekly = load_data("Weekly")
weekly_numerical = weekly.assign(Direction=(weekly["Direction"] == "Up").astype(int))
default = load_data("default")
default_numerical = default.assign(
    default=(default["default"] == "Yes").astype(int),
    student=(default["student"] == "Yes").astype(int),
)
boston = load_data("Boston")
sns.set_theme()

# Problem 1

$$
\begin{split}
p(X)&=\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}} \\
\frac{p(X)}{1-p(X)}&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{1-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1+e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1+e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1+e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}-\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}} \\
&=\frac{\frac{e^{\beta_0+\beta_1 X}}{1+e^{\beta_0+\beta_1 X}}}{\frac{1}{1+e^{\beta_0+\beta_1 X}}} \\
&=e^{\beta_0+\beta_1 X}
\end{split}
$$

# Problem 2

## Problem 2.a

On average, $\frac{1}{10}$ of the observations will be used to make the predictions.

## Problem 2.b

On average, $\frac{1}{100}$ of the observations will be used to make the predictions.

## Problem 2.c

On average, $10^{-100}$ of the observations will be used to make the predictions.

## Problem 2.d

As the number of features increases, fewer of the more extensive set will be used for inference, making the model seem relatively 'dumber' as it sees a very local picture and never really understands a larger picture, causing high variance and overfitting.

## Problem 2.e

$$
\begin{split}
s^{p}&=10^{-1} \\
p\ln s&=-\ln 10 \\
\ln s&=-p^{-1}\ln 10 \\
s&=10^{-p^{-1}}
\end{split}
$$

### $p=1$

$$
s=10^{-1}=0.1
$$

### $p=2$

$$
s=10^{-0.5}\approx .32
$$

### $p=100$

$$
s=10^{-.01}\approx 0.98
$$

# Problem 3

## Problem 3.a

$$
Y=\sigma\left(\hat{\beta}_0+\hat{\beta}_1 X_1+\hat{\beta}_2 X_2\right)=\sigma\left(-6+40\cdot .05+1\cdot 3.5\right)=\sigma(-0.5)\approx .377
$$

## Problem 3.b

$$
\begin{split}
0.5&=\sigma\left(\hat{\beta}_0+\hat{\beta}_1 X_1+\hat{\beta}_2 X_2\right)=\sigma\left(-6+0.05h+3.5\right) \\
0.5&=\sigma\left(0.05h-2.5\right) \\
h&\approx 50
\end{split}
$$

# Problem 4

We prefer the regression because the $K=1$ KNN will have a $0\%$ error on the training data ( the NN of any data point in the training data set will just be itself ), meaning it had a $36\%$ error on the testing data. Since the $36\%$ test error for KNN is more than the $30\%$ error on the linear regression, we would prefer the regression to classify new observations.

# Problem 5

## Problem 5.a

In [2]:
# | warning: false
sns.pairplot(weekly, hue="Direction", diag_kws={"fill": "True"})
plt.show()

/opt/homebrew/lib/python3.11/site-packages/seaborn/axisgrid.py:118: UserWarning:

The figure layout has changed to tight



<Figure size 6282.25x6000 with 72 Axes>

The graphs show that $\text{Today}$ is clustered by $\text{Direction}$, and there is a correlation between $\text{Volume}$ and $\text{Year}$.

## Problem 5.b

In [3]:
model = smf.logit(
    "Direction ~ Volume + Lag1 + Lag2 + Lag3 + Lag4 + Lag5", data=weekly_numerical
).fit(disp=False)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Direction   No. Observations:                 1089
Model:                          Logit   Df Residuals:                     1082
Method:                           MLE   Df Model:                            6
Date:                Fri, 28 Jul 2023   Pseudo R-squ.:                0.006580
Time:                        20:50:15   Log-Likelihood:                -743.18
converged:                       True   LL-Null:                       -748.10
Covariance Type:            nonrobust   LLR p-value:                    0.1313
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2669      0.086      3.106      0.002       0.098       0.435
Volume        -0.0227      0.037     -0.616      0.538      -0.095       0.050
Lag1          -0.0413      0.026     -1.563      0.118      -0.093       0.010
Lag2           0.0584      0.027      2.175      0.030       0.006       0.111
Lag3          -0.0161      0.027     -0.602      0.547      -0.068       0.036
Lag4          -0.0278      0.026     -1.050      0.294      -0.080       0.024
Lag5          -0.0145      0.026     -0.549      0.583      -0.066       0.037
==============================================================================
"""

$\text{Lag2}$ is the only statistically significant correlation.

## Problem 5.c

In [4]:
sns.set_theme(style="dark")
ConfusionMatrixDisplay.from_estimator(
    LogisticRegression().fit(
        weekly_numerical[["Volume", "Lag1", "Lag2", "Lag3", "Lag4", "Lag5"]],
        weekly_numerical["Direction"],
    ),
    weekly_numerical[["Volume", "Lag1", "Lag2", "Lag3", "Lag4", "Lag5"]],
    weekly_numerical["Direction"],
)
plt.show()

<Figure size 1650x1050 with 2 Axes>

We can see that the model is hesitant to guess $0$, so it will have a very high recall but a low precision and accuracy.

## Problem 5.d

In [5]:
weekly_training_numerical = weekly_numerical[weekly_numerical["Year"] <= 2008]
weekly_testing_numerical = weekly_numerical[weekly_numerical["Year"] >= 2009]
ConfusionMatrixDisplay.from_estimator(
    LogisticRegression().fit(
        weekly_training_numerical[["Lag2"]],
        weekly_training_numerical["Direction"],
    ),
    weekly_testing_numerical[["Lag2"]],
    weekly_testing_numerical["Direction"],
)
plt.show()

<Figure size 1650x1050 with 2 Axes>

We can see that the model is hesitant to guess $0$, so it will have a very high recall but a low precision and accuracy, just like the non-validation set counterpart, which means the model was not overfitting.

## Probelm 5.e

In [6]:
ConfusionMatrixDisplay.from_estimator(
    LinearDiscriminantAnalysis().fit(
        weekly_training_numerical[["Lag2"]],
        weekly_training_numerical["Direction"],
    ),
    weekly_testing_numerical[["Lag2"]],
    weekly_testing_numerical["Direction"],
)
plt.show()

<Figure size 1650x1050 with 2 Axes>

This is identical to the last matrix, so at least for this set, Linear Discriminant Analysis and Logistic Regression have identical results: very high recall but low precision and accuracy.

## Problem 5.f

In [7]:
ConfusionMatrixDisplay.from_estimator(
    QuadraticDiscriminantAnalysis().fit(
        weekly_training_numerical[["Lag2"]],
        weekly_training_numerical["Direction"],
    ),
    weekly_testing_numerical[["Lag2"]],
    weekly_testing_numerical["Direction"],
)
plt.show()

<Figure size 1650x1050 with 2 Axes>

This model never guesses $0$, which turns out to have a perfect recall but a lower precision and accuracy.

## Problem 5.g

In [8]:
ConfusionMatrixDisplay.from_estimator(
    KNeighborsClassifier(n_neighbors=1).fit(
        weekly_training_numerical[["Lag2"]],
        weekly_training_numerical["Direction"],
    ),
    weekly_testing_numerical[["Lag2"]],
    weekly_testing_numerical["Direction"],
)
plt.show()

<Figure size 1650x1050 with 2 Axes>

This is a very even split between the guesses and turns out to get slightly better precision and accuracy but a much lower recall.

## Probem 5.h

In [9]:
ConfusionMatrixDisplay.from_estimator(
    GaussianNB().fit(
        weekly_training_numerical[["Lag2"]],
        weekly_training_numerical["Direction"],
    ),
    weekly_testing_numerical[["Lag2"]],
    weekly_testing_numerical["Direction"],
)
plt.show()
sns.set_theme()

<Figure size 1650x1050 with 2 Axes>

This model also never guesses $0$, making it have a perfect recall but bad precision and accuracy.

# Problem 6

## Problem 6.a

Each bootstrap observation has a $\frac{1}{n}$ chance of being equal to the $j$th element of the original sample, so it has a $1-\frac{1}{n}=\frac{n-1}{n}$ chance of not being the $j$th observation.

## Problem 6.b

Every bootstrap observation is sampled identically, so it is also $\frac{n-1}{n}$

## Problem 6.c

Each bootstrap observation has a $1-\frac{1}{n}$ probability of not being the $j$th observation, so the chance that every one of $n$ bootstrap observations in a bootstrap sample of size $n$ is not the $j$th observation is $\left(1-\frac{1}{n}\right)^n$.

## Problem 6.d

$1-\left(1-\frac{1}{n}\right)^n=1-\left(1-\frac{1}{5}\right)^5\approx 0.672$

## Problem 6.e

$1-\left(1-\frac{1}{n}\right)^n=1-\left(1-\frac{1}{100}\right)^{100}\approx 0.634$

## Problem 6.f

$\left(1-\frac{1}{n}\right)^n=\left(1-\frac{1}{10000}\right)^{10000}\approx 0.632$

## Problem 6.g

In [10]:
x = np.arange(1, 10000)
sns.lineplot(1 - (1 - 1 / x) ** x)
plt.show()

<Figure size 1650x1050 with 1 Axes>

$\left(1-\frac{1}{n}\right)^n$ decreases and is a well known asymptote to $1-e^{-1}$.

## Python 6.h

In [11]:
rng = np.random.default_rng(10)
store = np.empty(10000)
for i in range(10000):
    store[i] = np.sum(rng.choice(100, replace=True) == 4) > 0

Markdown("$" + str(np.mean(store)) + "$")

$0.0089$

This is different from the estimate of $0.634$ from part e. However, I think the code provided does not accomplish what the quesiton intended. `np.sum(rng.choice(100, replace=True) == 4) > 0` seems to be trying to see if any of a sample of 100 bootstrap observations is $4$. However, it checks if one sample equals $4$, which explains the $\approx 1%$ answer. So instead, I am going to make some slight fixes to the code:

In [12]:
rng = np.random.default_rng(10)
store = np.empty(10000)
for i in range(10000):
    store[i] = np.sum(rng.choice(100, size=100, replace=True) == 4) > 0

Markdown("$" + str(np.mean(store)) + "$")

$0.6362$

And now it generates the correct value.

# Problem 7

## Probblem 7.a

In [13]:
smf.logit("default ~ income + balance", data=default_numerical).fit(
    disp=False
).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Fri, 28 Jul 2023   Pseudo R-squ.:                  0.4594
Time:                        20:50:16   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

## Problem 7.b

In [14]:
default_numerical_train, default_numerical_test = train_test_split(
    default_numerical, test_size=0.5, random_state=42
)
Markdown(
    "The error is $"
    + str(
        1
        - LogisticRegression()
        .fit(
            default_numerical_train[["income", "balance"]],
            default_numerical_train["default"],
        )
        .score(
            default_numerical_test[["income", "balance"]],
            default_numerical_test["default"],
        )
    )
    + "$"
)

The error is $0.032200000000000006$

## Problem 7.c

In [15]:
splits = [0.25, 0.5, 0.75]
errors = []
for split in splits:
    default_numerical_train, default_numerical_test = train_test_split(
        default_numerical, test_size=split, random_state=42
    )
    errors.append(
        1
        - LogisticRegression()
        .fit(
            default_numerical_train[["income", "balance"]],
            default_numerical_train["default"],
        )
        .score(
            default_numerical_test[["income", "balance"]],
            default_numerical_test["default"],
        )
    )
pd.DataFrame({"Training Split": splits, "Error": errors})

,Training Split,Error
0,0.25,0.027600
1,0.50,0.032200
2,0.75,0.033067


These errors are negligibly different and change significantly with a different seed.

## Problem 7.d

In [16]:
Markdown(
    "The error is $"
    + str(
        1
        - LogisticRegression()
        .fit(
            default_numerical_train[["student", "income", "balance"]],
            default_numerical_train["default"],
        )
        .score(
            default_numerical_test[["student", "income", "balance"]],
            default_numerical_test["default"],
        )
    )
    + "$"
)

The error is $0.03306666666666669$

This error is identical to the error without a student, which implies either that student is correlated with income or balance ( such that it does not provide new information ) or the student is not correlated with default.

# Problem 8

## Problem 8.a

In [17]:
model = smf.logit("default ~ income + balance", data=default_numerical).fit(disp=False)
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Fri, 28 Jul 2023   Pseudo R-squ.:                  0.4594
Time:                        20:50:16   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                4.541e-292
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.5405      0.435    -26.544      0.000     -12.393     -10.688
income      2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
balance        0.0056      0.000     24.835      0.000       0.005       0.006
==============================================================================

Possibly complete quasi-separation: A fraction 0.14 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

## Problem 8.b

In [18]:
def boot_fn(data, indexes):
    return (
        smf.logit("default ~ income + balance", data=data.loc[indexes])
        .fit(disp=False)
        .params
    )

## Problem 8.c

In [19]:
np.random.seed(42)
params = []
for i in range(1000):
    params.append(
        boot_fn(
            default_numerical,
            np.random.choice(default_numerical.index, len(default_numerical)),
        )
    )

In [20]:
pd.DataFrame(
    {"std error": np.std(params, axis=0)}, index=["Intercept", "income", "balance"]
)

,std error
Intercept,0.434501
income,0.000005
balance,0.000232


## Problem 8.d

The intercept and income std are nearly identical, while the balance std is very different ( $0$ in the prediction while small but nonzero in the bootstrapping ).

# Problem 9

## Problem 9.a

In [21]:
rng = np.random.default_rng(1)
x = rng.normal(size=100)
y = x - 2 * x**2 + rng.normal(size=100)

In this case, there are $100$ observations, so $n=100$, and there is one predictor ( $x$ ), so $p=1$. The equation is $$y = \beta_0 + \beta_1 x + \beta_2 x^2 + \epsilon$$ where $\beta_0=0$, $\beta_1=1$, $\beta_2=-2$.

## Problem 9.b

In [22]:
sns.scatterplot(x=x, y=y)
plt.show()

<Figure size 1650x1050 with 1 Axes>

It has a strong nonlinear relationship, which seems quadratic concave down.

## Problem 9.c

In [23]:
rng = np.random.default_rng(1)
pd.DataFrame(
    {
        "error": [
            -np.mean(
                cross_val_score(
                    LinearRegression(),
                    [[e**n for n in range(1, i + 1)] for e in x],
                    y,
                    cv=KFold(n_splits=len(y), shuffle=True, random_state=42),
                    scoring="neg_mean_squared_error",
                )
            )
            for i in range(1, 5)
        ]
    },
    index=["Degree {}".format(x) for x in range(1, 5)],
)

,error
Degree 1,6.633030
Degree 2,1.122937
Degree 3,1.301797
Degree 4,1.332394


## Problem 9.d

In [24]:
pd.DataFrame(
    {
        "error": [
            -np.mean(
                cross_val_score(
                    LinearRegression(),
                    [[e**n for n in range(1, i + 1)] for e in x],
                    y,
                    cv=KFold(n_splits=len(y), shuffle=True, random_state=1),
                    scoring="neg_mean_squared_error",
                )
            )
            for i in range(1, 5)
        ]
    },
    index=["Degree {}".format(x) for x in range(1, 5)],
)

,error
Degree 1,6.633030
Degree 2,1.122937
Degree 3,1.301797
Degree 4,1.332394


The numbers are identical, which is expected because the random seed only changes the order in which elements are left out. However, at some point, every element will be left out once, so the values for the error will still be identical, just in a different order. Therefore, the mean of the values will also be identical.

## Problem 9.e

The second-degree polynomial performs best in both tests, which is what we expect; we expect a smaller degree to underfit and a larger degree to overfit.

## Problem 9.f

In [25]:
smf.ols("y ~ x", data=pd.DataFrame({"x": x, "y": y})).fit(disp=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.311
Method:                 Least Squares   F-statistic:                     45.60
Date:                Fri, 28 Jul 2023   Prob (F-statistic):           1.04e-09
Time:                        20:50:27   Log-Likelihood:                -230.83
No. Observations:                 100   AIC:                             465.7
Df Residuals:                      98   BIC:                             470.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.4650      0.247     -5.937      0.000      -1.955      -0.975
x              1.9494      0.289      6.752      0.000       1.376       2.522
==============================================================================
Omnibus:                       52.788   Durbin-Watson:                   1.972
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              149.089
Skew:                          -1.953   Prob(JB):                     4.22e-33
Kurtosis:                       7.530   Cond. No.                         1.20
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
smf.ols("y ~ x + np.power(x,2)", data=pd.DataFrame({"x": x, "y": y})).fit(
    disp=False
).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.884
Method:                 Least Squares   F-statistic:                     379.5
Date:                Fri, 28 Jul 2023   Prob (F-statistic):           1.36e-46
Time:                        20:50:27   Log-Likelihood:                -141.06
No. Observations:                 100   AIC:                             288.1
Df Residuals:                      97   BIC:                             295.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0728      0.119     -0.611      0.543      -0.309       0.164
x                  0.9663      0.126      7.647      0.000       0.715       1.217
np.power(x, 2)    -2.0047      0.091    -22.072      0.000      -2.185      -1.824
==============================================================================
Omnibus:                        1.338   Durbin-Watson:                   2.197
Prob(Omnibus):                  0.512   Jarque-Bera (JB):                0.814
Skew:                           0.119   Prob(JB):                        0.666
Kurtosis:                       3.372   Cond. No.                         2.23
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [27]:
smf.ols(
    "y ~ x + np.power(x,2) + np.power(x,3)", data=pd.DataFrame({"x": x, "y": y})
).fit(disp=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.885
Method:                 Least Squares   F-statistic:                     253.8
Date:                Fri, 28 Jul 2023   Prob (F-statistic):           1.70e-45
Time:                        20:50:27   Log-Likelihood:                -140.47
No. Observations:                 100   AIC:                             288.9
Df Residuals:                      96   BIC:                             299.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0572      0.120     -0.477      0.635      -0.295       0.181
x                  1.1146      0.187      5.945      0.000       0.742       1.487
np.power(x, 2)    -2.0471      0.099    -20.673      0.000      -2.244      -1.851
np.power(x, 3)    -0.0643      0.060     -1.070      0.287      -0.184       0.055
==============================================================================
Omnibus:                        0.845   Durbin-Watson:                   2.199
Prob(Omnibus):                  0.655   Jarque-Bera (JB):                0.392
Skew:                           0.052   Prob(JB):                        0.822
Kurtosis:                       3.289   Cond. No.                         5.95
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
smf.ols(
    "y ~ x + np.power(x,2) + np.power(x,3) + np.power(x,4)",
    data=pd.DataFrame({"x": x, "y": y}),
).fit(disp=False).summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.890
Method:                 Least Squares   F-statistic:                     200.2
Date:                Fri, 28 Jul 2023   Prob (F-statistic):           2.22e-45
Time:                        20:50:27   Log-Likelihood:                -137.74
No. Observations:                 100   AIC:                             285.5
Df Residuals:                      95   BIC:                             298.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.1008      0.136      0.743      0.460      -0.169       0.370
x                  0.9050      0.205      4.423      0.000       0.499       1.311
np.power(x, 2)    -2.5059      0.221    -11.336      0.000      -2.945      -2.067
np.power(x, 3)     0.0338      0.073      0.466      0.642      -0.110       0.178
np.power(x, 4)     0.1042      0.045      2.309      0.023       0.015       0.194
==============================================================================
Omnibus:                        2.476   Durbin-Watson:                   2.163
Prob(Omnibus):                  0.290   Jarque-Bera (JB):                2.097
Skew:                           0.118   Prob(JB):                        0.351
Kurtosis:                       3.669   Cond. No.                         19.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Yes, the statistical significance results make sense because we see a strong correlation with the $0$th and $1$st degree terms, which are a part of the known function for $y$, and a not statistically significant correlation with the $2$nd and $3$rd degree polynomials because they not a part of the know function for $y$. This also agrees with the results from the cross-validation tests: that $3$rd and $4$th-degree terms are not helpful/harmful.

# Problem 10
## Problem 10.a

In [29]:
mean = np.mean(boston["medv"])
Markdown("The mean is $" + str(mean) + "$")

The mean is $22.532806324110677$

## Problem 10.b

In [30]:
sem = np.std(boston["medv"]) / np.sqrt(np.size(boston["medv"]))
Markdown("The standard error is $" + str(sem) + "$")

The standard error is $0.4084569346972866$

## Problem 10.c

In [31]:
np.random.seed(42)
boostrap = [
    np.mean(np.random.choice(boston["medv"], size=len(boston["medv"]), replace=True))
    for i in range(1000)
]
Markdown("The standard error is $" + str(np.std(boostrap)) + "$")

The standard error is $0.39688532048705005$

The bootstrapped estimation is nearly identical to the calculated estimation

## Problem 10.d

In [32]:
Markdown(
    "The boostrap confidence interval is $("
    + str(np.percentile(boostrap, 5))
    + ", "
    + str(np.percentile(boostrap, 95))
    + ")$"
)

The boostrap confidence interval is $(21.89533596837945, 23.170207509881426)$

In [33]:
Markdown(
    "The standard error confidence interval is $("
    + str(mean - 2 * sem)
    + ", "
    + str(mean + 2 * sem)
    + ")$"
)

The standard error confidence interval is $(21.715892454716105, 23.34972019350525)$

These are once again nearly identical

## Problem 10.e

In [34]:
Markdown("The median is $" + str(np.median(boston["medv"])) + "$")

The median is $21.2$

## Problem 10.f

In [35]:
Markdown(
    "The standard error of the median is $"
    + str(
        np.std(
            [
                np.median(
                    np.random.choice(
                        boston["medv"], size=len(boston["medv"]), replace=True
                    )
                )
                for i in range(1000)
            ]
        )
    )
    + "$"
)

The standard error of the median is $0.3758688468069677$

This makes sense given the data: it is around the standard error of the mean.

## Problem 10.g

In [36]:
Markdown("The 10th percentile is $" + str(np.percentile(boston["medv"], 0.1)) + "$")

The 10th percentile is $5.0$

## Problem 10.h

In [37]:
Markdown(
    "The standard error of the 10th percentile is $"
    + str(
        np.std(
            [
                np.percentile(
                    np.random.choice(
                        boston["medv"], size=len(boston["medv"]), replace=True
                    ),
                    0.1,
                )
                for i in range(1000)
            ]
        )
    )
    + "$"
)

The standard error of the 10th percentile is $0.4769626719961951$

The standard error of the $10$th percentile passes the sanity check; it is also similar to the standard error of the mean and median and is not very large or small compared to the data.